## Toy problem

In [1]:
import torch
import numpy

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('Running on the GPU.')
else:
    device = torch.device('cpu')
    print('Running on the CPU.')

Running on the GPU.


In [3]:
models_ids = ['A', 'B', 'C'] 
models = numpy.asarray([[43], [-828], [453]], dtype = float)
data = {}
data_map = []
data_length = 0
for idx, model_id in enumerate(models_ids):
    data[model_id] = {}
    data[model_id]['model_id'] = model_id
    data[model_id]['model'] = models[idx]
    data[model_id]['vectors'] = [numpy.random.normal(data[model_id]['model'][0], 5.0, 1) for i in range(10000)]
    length = len(data[model_id]['vectors'])
    data_map.append((data_length, data_length + length - 1, model_id))
    data_length += length
print(data_map)

[(0, 9999, 'A'), (10000, 19999, 'B'), (20000, 29999, 'C')]


In [7]:
from torch.utils.data import Dataset, DataLoader, random_split

class Toy_dataset(Dataset):
    def __init__(self, data, data_length, data_map):
        self.data = data
        self.data_length = data_length
        self.data_map = data_map
    def __len__(self):
        return self.data_length
    def __getitem__(self, idx):
        data_tuple = list(filter(lambda data_tuple: data_tuple[0] <= idx and idx <= data_tuple[1], self.data_map))[0]
        data_idx = idx - data_tuple[0]
        data_id = data_tuple[2]
        entry = data[data_id]
        vector = entry['vectors'][data_idx]
        permutations = [model_id for model_id in data]
        models = [data[model_id]['model'] for model_id in permutations]
        permutations = numpy.asarray([model_id == entry['model_id'] for model_id in permutations], dtype = float)
        
        x = numpy.concatenate([vector] + models)
        y = permutations
        
        return x, y
        
toy_dataset = Toy_dataset(data, data_length, data_map)

train_length = int(len(toy_dataset) * 0.7)
test_length = len(toy_dataset) - train_length
cross_length = int(train_length * 0.3)
train_length = train_length - cross_length

train_dataset, cross_dataset, test_dataset = random_split(toy_dataset, [train_length, cross_length, test_length])
train_dataloader = DataLoader(train_dataset, batch_size = 10, shuffle=True, num_workers = 4)
cross_dataloader = DataLoader(cross_dataset, batch_size = len(cross_dataset), num_workers = 4)
test_dataloader = DataLoader(test_dataset, batch_size = len(test_dataset), num_workers = 4)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc0 = nn.Linear(4, 4)
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)
    def forward(self, x):
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

In [9]:
import torch.optim as optim

net = Net().to(device)

optimizer = optim.Adam(net.parameters(), lr = 0.001)
criterion = nn.BCELoss()

epochs = 10
for epoch in range(epochs):
    for input, target in train_dataloader:
        input = input.to(device, non_blocking=True).float()
        target = target.to(device, non_blocking=True).float()

        net.zero_grad()

        output = net(input)

        loss = criterion(output, target)

        loss.backward()
        optimizer.step()
    with torch.no_grad():
        for input, target in cross_dataloader:
            input = input.to(device, non_blocking=True).float()
            target = target.to(device, non_blocking=True).float()
            output = net(input)
            cross_loss = criterion(output, target)
        
    print('epoch:', epoch, 'loss:', loss, 'cross_loss:', cross_loss)
    
    if cross_loss < 0.01:
        print('Done training.')
        break

epoch: 0 loss: tensor(0.1842, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) cross_loss: tensor(0.1367, device='cuda:0')
epoch: 1 loss: tensor(0.0511, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) cross_loss: tensor(0.0547, device='cuda:0')
epoch: 2 loss: tensor(0.0100, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) cross_loss: tensor(0.0090, device='cuda:0')
Done training.


In [10]:
with torch.no_grad():
    for input, target in test_dataloader:
        input = input.to(device, non_blocking=True).float()
        target = target.to(device, non_blocking=True).float()
        output = net(input)
        test_loss = criterion(output, target)
    print('test_loss:', test_loss)

test_loss: tensor(0.0091, device='cuda:0')
